In [ ]:
import numpy as np
import pandas as pd
import json
import os
from PIL import Image

import matplotlib.pyplot as plt
from tqdm import tqdm

%matplotlib inline

In [ ]:
def load_json(jfile):
    with open(jfile, 'rb') as f:
        return json.load(f)

In [ ]:
PATH = '/media/nas/BasesDeDonnees/cocodataset/'
#path = '../cocodataset/'

#train_2014 = PATH+'train_2014'
#valid_2014 = PATH+'valid_2014'
#test_2014 = PATH+'test_2014'

#test_2015 = PATH+'test_2015'

train_2017 = PATH+'train_2017'
valid_2017 = PATH+'valid_2017'
#test_2017 = PATH+'test_2017'

#train_2014 = PATH+'annotations/instances_train2017.json'
#valid_2014 = PATH+'annotations/instances_val2017.json'
#test_2014 = PATH+'annotations/image_nfi_test2017.json'

train_2017 = PATH+'annotations/instances_train2017.json'
valid_2017 = PATH+'annotations/instances_val2017.json'
#test_2017 = PATH+'annotations/image_nfi_test2017.json'

#train_2014_data = load_json(train_2014)
#valid_2014_data = load_json(valid_2014)
train_2017_data = load_json(train_2017)
valid_2017_data = load_json(valid_2017)

In [ ]:
import os

paths = ['COCO1',
         'COCO1/train',
         'COCO1/train/images',
         'COCO1/train/labels',
         'COCO1/valid',
         'COCO1/valid/images',
         'COCO1/valid/labels',
         'COCO1/support',
         'COCO1/support/images']

        #  'COCO2',
        #  'COCO2/train',
        #  'COCO2/train/images',
        #  'COCO2/train/labels',
        #  'COCO2/valid',
        #  'COCO2/valid/images',
        #  'COCO2/valid/labels',
        #  'COCO2/support',
        #  'COCO2/support/images',
        #  'COCO3',
        #  'COCO3/train',
        #  'COCO3/train/images',
        #  'COCO3/train/labels',
        #  'COCO3/valid',
        #  'COCO3/valid/images',
        #  'COCO3/valid/labels',
        #  'COCO3/support',
        #  'COCO3/support/images',
        #  'COCO4',
        #  'COCO4/train',
        #  'COCO4/train/images',
        #  'COCO4/train/labels',
        #  'COCO4/valid',
        #  'COCO4/valid/images',
        #  'COCO4/valid/labels',
        #  'COCO4/support',
        #  'COCO4/support/images']

for path in paths:
    if not os.path.exists(path):
        os.mkdir(path)

In [ ]:
train_2017_data.keys()

In [ ]:
images_df = pd.DataFrame(train_2017_data['images'])
anns_df = pd.DataFrame(train_2017_data['annotations'])
category = pd.DataFrame(train_2017_data['categories'])

all_train = pd.merge(anns_df, images_df, left_on='image_id', right_on='id')

images_df = pd.DataFrame(valid_2017_data['images'])
anns_df = pd.DataFrame(valid_2017_data['annotations'])
category = pd.DataFrame(valid_2017_data['categories'])

all_valid = pd.merge(anns_df, images_df, left_on='image_id', right_on='id')

In [ ]:
anns_df['category_id'].unique()

In [ ]:
all_valid.head()

# PROCESSING CLasses


In [ ]:
print(all_train['category_id'].unique().shape)
print(all_valid['category_id'].unique().shape)

In [ ]:
coco_classes = all_train['category_id'].unique()
split1 = coco_classes[[(i-1) for i in [1,5,9,13,17,21,25,29,33,37,41,45,49,53,57,61,65,69,73,77]]]
#split2 = coco_classes[[(i-1) for i in [2,6,10,14,18,22,26,30,34,38,42,46,50,54,58,62,66,70,74,78]]]
#split3 = coco_classes[[(i-1) for i in [3,7,11,15,19,23,27,31,35,39,43,47,51,55,59,63,67,71,75,79]]]
#split4 =coco_classes[[(i-1) for i in  [4,8,12,16,20,24,28,32,36,40,44,48,52,56,60,64,68,72,76,80]]]

print(len(split1))#,len(split2),len(split3),len(split4))

In [ ]:
split1_train = all_train[-all_train['category_id'].isin(split1)]
#split2_train = all_train[-all_train['category_id'].isin(split2)]
#split3_train = all_train[-all_train['category_id'].isin(split3)]
#split4_train = all_train[-all_train['category_id'].isin(split4)]

split1_valid = all_valid[all_valid['category_id'].isin(split1)]
#split2_valid = all_valid[all_valid['category_id'].isin(split2)]
#split3_valid = all_valid[all_valid['category_id'].isin(split3)]
#split4_valid = all_valid[all_valid['category_id'].isin(split4)]


# PROCESSING JSON

In [ ]:
print(split1_train['category_id'].unique().shape)
print(split1_valid['category_id'].unique().shape)

# print(split2_train['category_id'].unique().shape)
# print(split2_valid['category_id'].unique().shape)

# print(split3_train['category_id'].unique().shape)
# print(split3_valid['category_id'].unique().shape)

# print(split4_train['category_id'].unique().shape)
# print(split4_valid['category_id'].unique().shape)

In [ ]:
category['name'].unique()

In [ ]:
plt.figure()

label = np.arange(len(split1_valid['category_id'].unique()))
l = [(i-1) for i in [1,5,9,13,17,21,25,29,33,37,41,45,49,53,57,61,65,69,73,77]]
names = category['name'].unique()[l]

plt.barh(label, split1_valid['category_id'].value_counts())
plt.yticks(label, names)


In [ ]:
len(split1_train['file_name'].unique())

# Base on DANA support IMAGE

In [ ]:
#!wget -O support.zip "https://drive.google.com/uc?export=download&id=1nl9-DEpBBJ5w6hxVdijY6hFxoQdz8aso&confirm=yes"

In [ ]:
#!unzip support.zip

In [ ]:
category.head()

In [ ]:
from PIL import Image, ImageOps

def resize_with_padding(img, expected_size):
    width, height = img.size
    ratio = width / height
    if width < height:
        img = img.resize((int(ratio*expected_size[0]),int(expected_size[0])))
    else:
        img = img.resize((int(expected_size[1]),int((1/ratio)*expected_size[1])))
        
    img.thumbnail((expected_size[0], expected_size[1]))
    delta_width = expected_size[0] - img.size[0]
    delta_height = expected_size[1] - img.size[1]
    pad_width = delta_width // 2
    pad_height = delta_height // 2
    padding = (pad_width, pad_height, delta_width - pad_width, delta_height - pad_height)
    return ImageOps.expand(img, padding)


In [ ]:
#TODO support for train:
import glob

def saveSupport(path):
    for row in tqdm(category.iloc(),total=len(category)):
        name = row['name']
        c = row['id']
        liste = glob.glob('supports/all/'+name+'/*')
        f = liste[0]
        image = Image.open(f)
        image = resize_with_padding(image, (640,640))
        image.save(path+str(c)+'.png')

In [ ]:
#saveSupport('COCO1/support/images/')

In [ ]:
def saveImage(pd, folder, path):
    list_image = pd['file_name'].unique()
    for name in tqdm(list_image):
        image = Image.open(PATH+folder+name)
        for c in pd.loc[pd['file_name'] == name]['category_id'].unique():
            image = resize_with_padding(image, (640,640))
            image.save(path+name[:-4]+'.'+str(c)+'.png')
        

In [ ]:
#saveImage(split1_train,'train2017/','COCO1/train/images/')
#saveImage(split1_valid,'val2017/','COCO1/valid/images/')

In [ ]:
# saveImage(split2_train,'train2017/','COCO2/train/images/')
# saveImage(split2_valid,'val2017/','COCO2/valid/images/')

In [ ]:
# saveImage(split3_train,'train2017/','COCO3/train/images/')
# saveImage(split3_valid,'val2017/','COCO3/valid/images/')

In [ ]:
# saveImage(split4_train,'train2017/','COCO4/train/images/')
# saveImage(split4_valid,'val2017/','COCO4/valid/images/')

In [ ]:
def resize_label_with_padding(old_size, expected_size, bbox):
    (x,y,w,h) = bbox

    ratio = old_size[0] / old_size[1]
    if old_size[0] < old_size[1]:
        new = (int(ratio*expected_size[0]),int(expected_size[0]))
    else:
        new = (int(expected_size[1]),int((1/ratio)*expected_size[1]))

    delta_width = expected_size[0] - new[0]
    delta_height = expected_size[1] - new[1]

    pad_width = delta_width // 2
    pad_height = delta_height // 2

    x_t = (pad_width +  (x/old_size[0])*new[0] + ((w/old_size[0])*new[0])/2 ) / expected_size[0]
    y_t = (pad_height + (y/old_size[1])*new[1] + ((h/old_size[1])*new[1])/2) / expected_size[1]
    w_t = (w/old_size[0])*new[0] / expected_size[0]
    h_t = (h/old_size[1])*new[1] / expected_size[1]
    
    return x_t,y_t,w_t,h_t

In [ ]:
def savetxt(pd, path):
    list_image = pd['file_name'].unique()
    for name in tqdm(list_image):
        for c in pd.loc[pd['file_name'] == name]['category_id'].unique():
            annotation = pd.loc[pd['file_name'] == name]
            annotation = annotation[annotation['category_id'] == c]
            bbox = annotation['bbox']

            with open(path+name[:-4]+'.'+str(c)+'.txt', 'wt') as f:
                for box,width,height in zip(bbox,annotation['width'],annotation['height']):
                    x,y,w,h = resize_label_with_padding((width,height), (640,640), box)
                    s = '0'+' '+str(x)+' '+str(y)+' '+str(w)+' '+str(h)+' \n'
                    f.write(s)

In [ ]:
savetxt(split1_train,'COCO1/train/labels/')
savetxt(split1_valid,'COCO1/valid/labels/')

In [ ]:
# savetxt(split2_train,'COCO2/train/labels/')
# savetxt(split2_valid,'COCO2/valid/labels/')

In [ ]:
# savetxt(split3_train,'COCO3/train/labels/')
# savetxt(split3_valid,'COCO3/valid/labels/')

In [ ]:
# savetxt(split4_train,'COCO4/train/labels/')
# savetxt(split4_valid,'COCO4/valid/labels/')

In [ ]:
with open('COCO1/data.yaml', 'a') as f:
    f.write('train: ../train/images \n')
    f.write('val: ../valid/images \n')
    f.write('support: ../support/images \n')
    f.write(' \n')
    f.write('nc: 1 \n')
    f.write('names: [\'object\']')

In [ ]:
# with open('COCO2/data.yaml', 'a') as f:
#     f.write('train: ../train/images \n')
#     f.write('val: ../valid/images \n')
#     f.write('support: ../support/images \n')
#     f.write(' \n')
#     f.write('nc: 1 \n')
#     f.write('names: [\'object\']')

In [ ]:
# with open('COCO3/data.yaml', 'a') as f:
#     f.write('train: ../train/images \n')
#     f.write('val: ../valid/images \n')
#     f.write('support: ../support/images \n')
#     f.write(' \n')
#     f.write('nc: 1 \n')
#     f.write('names: [\'object\']')

In [ ]:
# with open('COCO4/data.yaml', 'a') as f:
#     f.write('train: ../train/images \n')
#     f.write('val: ../valid/images \n')
#     f.write('support: ../support/images \n')
#     f.write(' \n')
#     f.write('nc: 1 \n')
#     f.write('names: [\'object\']')